In [ ]:
#| default_exp bashxtract

In [ ]:
#| export
import shlex,subprocess,json,shutil
from fastcore.utils import *

# Bashxtract API
> Extract commands used from bash command lines

## Introduction

`safecmd.bashxtract` provides tools for parsing and extracting commands from bash command strings. It's designed for security-conscious applications where you need to understand exactly what commands a shell script will execute before running it.

The core use case is validating shell commands from untrusted sources (like LLM-generated commands) against an allowlist. Rather than trying to regex-match bash syntax—which is notoriously tricky—this module uses `shfmt`, a proper bash parser, to build an AST and then extracts all executable commands from it.

It's likely that the only function you'll actually need from here is `extract_commands`. But we provide a full API of all the pieces we use to build that function, which we'll take you through here.

## The Problem

Parsing bash commands is surprisingly tricky. You might think `shlex.split` would work:

In [ ]:
cmd = '''
echo | head 2 <<EOF
asdf
jkljl
EOF
'''

In [ ]:
shlex.split(cmd)

['echo', '|', 'head', '2', '<<EOF', 'asdf', 'jkljl', 'EOF']

But `shlex` doesn't understand bash syntax—it treats `|`, `<<EOF`, and the heredoc content as regular arguments. It can't tell us that `echo` and `head` are separate commands in a pipeline, or that the heredoc content is input, not an argument.

Fortunately, there's a proper bash parser called `shfmt` that can parse bash into a JSON AST (Abstract Syntax Tree). The `shfmt-py` package (installed automatically with safecmd) provides the binary:

In [ ]:
!shfmt --help 2>&1 | head -6

usage: shfmt [flags] [path ...]

shfmt formats shell programs. If the only argument is a dash ('-') or no
arguments are given, standard input will be used. If a given path is a
directory, all shell scripts found under that directory will be used.



…and this is the key flag that we will use:

In [ ]:
!shfmt --help 2>&1 | grep to-json

  --to-json           print syntax tree to stdout as a typed JSON


As we'll see below, the AST that `shfmt` creates represents the structure of the bash command as nested dictionaries. Each node has a `Type` field telling us what kind of construct it is—`CallExpr` for a command invocation, `BinaryCmd` for pipelines and logical operators, `Word` for arguments, and so on. From this, we can pull out just the information we need.

For our heredoc example, what we ultimately want is to extract:

1. **The commands**: `['echo']` and `['head', '2']`
2. **The operators used**: `{'|'}` (a pipe)
3. **The heredoc content** attached to the command that receives it

So our goal is to walk this AST and produce a simple list of commands with their arguments, plus a set of operators—something we can easily validate against an allowlist.

The module is structured in layers:

1. **Parsing layer** (`parse_bash`): Converts bash syntax to a JSON AST using shfmt
2. **Text extraction** (`part_text`, `word_text`): Reconstructs text values from AST nodes, handling quotes, escapes, and expansions
3. **AST walking** (`visit_stmts`, `nested_stmts`): Recursively traverses the AST to find all commands, including those nested in substitutions
4. **Operator detection** (`collect_ops`): Identifies shell operators like pipes, redirects, and logical operators
5. **Validation** (`check_types`): Ensures we only process bash constructs we understand
6. **Main API** (`extract_commands`): Combines everything into a simple interface

## Parsing

In [ ]:
#| export
def parse_bash(cmd:str, shfmt:str='shfmt'):
    "Parse `cmd` using `shfmt`"
    if not shutil.which(shfmt):
        raise FileNotFoundError(f"{shfmt} not found in PATH. If you installed safecmd with --user, add ~/.local/bin to your PATH.")
    res = subprocess.run([shfmt, '--to-json'], input=cmd, capture_output=True, text=True)
    if res.returncode != 0: raise ValueError(res.stderr)
    return json.loads(res.stdout)

Parses a bash command string using `shfmt --to-json` and returns the AST as a Python dict. Raises `ValueError` if the command has syntax errors. Requires the `shfmt` binary to be available.

In [ ]:
parse_bash('echo hello')

{'Type': 'File',
 'Pos': {'Offset': 0, 'Line': 1, 'Col': 1},
 'End': {'Offset': 10, 'Line': 1, 'Col': 11},
 'Stmts': [{'Pos': {'Offset': 0, 'Line': 1, 'Col': 1},
   'End': {'Offset': 10, 'Line': 1, 'Col': 11},
   'Cmd': {'Type': 'CallExpr',
    'Pos': {'Offset': 0, 'Line': 1, 'Col': 1},
    'End': {'Offset': 10, 'Line': 1, 'Col': 11},
    'Args': [{'Pos': {'Offset': 0, 'Line': 1, 'Col': 1},
      'End': {'Offset': 4, 'Line': 1, 'Col': 5},
      'Parts': [{'Type': 'Lit',
        'Pos': {'Offset': 0, 'Line': 1, 'Col': 1},
        'End': {'Offset': 4, 'Line': 1, 'Col': 5},
        'ValuePos': {'Offset': 0, 'Line': 1, 'Col': 1},
        'ValueEnd': {'Offset': 4, 'Line': 1, 'Col': 5},
        'Value': 'echo'}]},
     {'Pos': {'Offset': 5, 'Line': 1, 'Col': 6},
      'End': {'Offset': 10, 'Line': 1, 'Col': 11},
      'Parts': [{'Type': 'Lit',
        'Pos': {'Offset': 5, 'Line': 1, 'Col': 6},
        'End': {'Offset': 10, 'Line': 1, 'Col': 11},
        'ValuePos': {'Offset': 5, 'Line': 1, 'C

## Text Extraction

In [ ]:
#| export
def part_text(p, cmd):
    "Extracts the text value from a single word part node in the shfmt AST."
    t = p.get('Type', '')
    if t == 'Lit': return p['Value'].replace('\\ ', ' ')
    if t == 'SglQuoted': return p['Value']
    if t in ('DblQuoted', 'Hdoc'): return ''.join(part_text(x, cmd) for x in p.get('Parts', []))
    if t == 'ParamExp':
        name = p['Param']['Value']
        if p.get('Index'): name += '[' + word_text(p['Index'], cmd) + ']'
        return '${' + name + '}' if p.get('Rbrace') else '$' + name
    if t in ('CmdSubst', 'ProcSubst'):
        start, end = p['Pos']['Offset'], p['End']['Offset']
        return cmd[start:end]
    return ''

Handles literals (with backslash-space unescaping), single/double quoted strings, parameter expansions (`$var`, `${var}`, `${arr[0]}`), and command/process substitutions. For substitutions, returns the original source text using offset positions.

In [ ]:
part_text({'Type': 'SglQuoted', 'Value': 'foo bar'}, "echo 'foo bar'")

'foo bar'

In [ ]:
#| export
def word_text(w, cmd):
    "Converts a Word node (with `Parts`) into its full text repr by concatenating `part_text` for each part."
    return ''.join(part_text(p, cmd) for p in w.get('Parts', []))

In [ ]:
word_text({'Parts': [{'Type': 'Lit', 'Value': 'hello'}]}, 'echo hello')

'hello'

In [ ]:
#| export
def nested_stmts(parts):
    "Yield all Stmts lists from nested Parts recursively"
    for p in parts:
        if stmts := p.get('Stmts'): yield stmts
        yield from nested_stmts(p.get('Parts', []))

Generator that recursively yields all `Stmts` lists found within nested `Parts` arrays. Used to find command substitutions (`$(...)`) and process substitutions (`<(...)`) at any nesting depth, including those inside double-quoted strings.

## AST Walking

In [ ]:
parts = [{'Type': 'CmdSubst', 'Stmts': [{'Cmd': {...}}]}]
list(nested_stmts(parts))

[[{'Cmd': {Ellipsis}}]]

In [ ]:
#| export
def visit_stmts(stmts, cmd, commands=None):
    "Visit statements, appending commands and handling redirects"
    if commands is None: commands=[]
    def visit(n):
        if not n: return
        if args := n.get('Args'):
            commands.append([word_text(a, cmd) for a in args])
            for a in args:
                for s in nested_stmts(a.get('Parts', [])): visit_stmts(s, cmd, commands)
        for k in ('Cmd', 'X', 'Y', 'Cond', 'Then', 'Else', 'Do', 'Loop'): visit(n.get(k))
        for k in ('Stmts', 'Items', 'Cases'): visit_stmts(n.get(k), cmd, commands)
    for s in stmts or []:
        visit(s)
        for r in s.get('Redirs', []):
            if not commands: continue
            if hdoc := r.get('Hdoc'): commands[-1].append(word_text(hdoc, cmd).rstrip('\n'))
            elif r.get('Op') == 63: commands[-1].extend(['<<<', word_text(r['Word'], cmd)])
    return commands

Walks a list of statement nodes from the shfmt AST, extracting all commands (including nested ones) into the `commands` list. Each command is represented as `[cmd, arg1, arg2, ...]`. Handles redirects by appending heredoc/here-string content to the most recent command. Returns the commands list.

In [ ]:
parsed = parse_bash('echo foo; cat file')
visit_stmts(parsed['Stmts'], 'echo foo; cat file')

[['echo', 'foo'], ['cat', 'file']]

In [ ]:
#| export
_op_map = {10: '&&', 11: '||', 12: '|', 13: '|&', 54: '>', 55: '>>', 56: '<', 58: '<&', 59: '>&', 64: '&>', 65: '&>>'}
_attr_ops = {'Background': '&', 'Semicolon': ';', 'Assigns': '='}

def collect_ops(node, ops=None):
    "Walk AST node and collect all operators into a set"
    if ops is None: ops = set()
    if not isinstance(node, dict): return ops
    for attr, op in _attr_ops.items():
        if node.get(attr): ops.add(op)
    if op := _op_map.get(node.get('Op')): ops.add(op)
    for v in node.values(): collect_ops(v, ops) if isinstance(v, dict) else [collect_ops(x, ops) for x in v] if isinstance(v, list) else None
    return ops

Detects `&`, `;`, logical AND/OR (`&&`/`||`), pipe (`|`), and redirections (`>`, `>>`, `<`) by checking the `Op` field against `OP_MAP` and the `Background`/`Semicolon` boolean flags.

In [ ]:
collect_ops(parse_bash('echo a && echo b | cat > out.txt'))

{'&&', '>', '|'}

In [ ]:
#| export
HANDLED_TYPES = {'File', 'CallExpr', 'BinaryCmd', 'Subshell', 'Block', 'IfClause', 'WhileClause', 
    'ForClause', 'CaseClause', 'CaseItem', 'FuncDecl', 'DeclClause', 'Lit', 'SglQuoted', 'DblQuoted',
    'ParamExp', 'CmdSubst', 'ProcSubst', 'Hdoc', 'Word', 'Redirect', 'Comment', 'ArithmExp', 'ArithmCmd'}

def check_types(node):
    "Raise ValueError if AST contains unhandled node types"
    if not isinstance(node, dict): return
    if (t := node.get('Type')) and t not in HANDLED_TYPES: raise ValueError(f"Unhandled bash construct: {t}")
    for v in node.values():
        if isinstance(v, dict): check_types(v)
        elif isinstance(v, list): [check_types(x) for x in v]

Raises `ValueError` if any node has a `Type` not in `HANDLED_TYPES`. Use this to detect unsupported bash constructs early, ensuring the rest of the parsing pipeline won't silently skip or mishandle unknown syntax.

In [ ]:
check_types(parse_bash('echo hello'))

In [ ]:
try: check_types(parse_bash('[[ -f foo ]]'))
except ValueError: print('Caught unhandled construct')

Caught unhandled construct


## Main API

In [ ]:
#| export
def extract_commands(cmd, shfmt='shfmt'):
    "Split bash command into (commands, operators)"
    ast = parse_bash(cmd, shfmt=shfmt)
    check_types(ast)
    return visit_stmts(ast.get('Stmts', []), cmd), collect_ops(ast)

`extract_commands(cmd)` parses a bash command string and returns a list of all commands that would be executed, including those embedded in command substitutions, process substitutions, pipelines, and compound statements.

Each command is represented as a list of tokens (strings), similar to the output of `shlex.split()`. The function uses `shfmt` to parse the bash syntax into an AST, then extracts all executable commands recursively.

**Handled constructs:**
- Simple commands: `echo foo` → `[['echo', 'foo']]`
- Pipelines: `cat file | grep x` → `[['cat', 'file'], ['grep', 'x']]`
- Sequences (`;`, `&`, `&&`, `||`): `echo a; echo b` → `[['echo', 'a'], ['echo', 'b']]`
- Command substitution: `echo $(whoami)` → `[['echo', '$(whoami)'], ['whoami']]`
- Backtick substitution: `` echo `whoami` `` → `[['echo', '`whoami`'], ['whoami']]`
- Process substitution: `diff <(ls a) <(ls b)` → `[['diff', ...], ['ls', 'a'], ['ls', 'b']]`
- Subshells: `(cd /tmp && rm *)` → `[['cd', '/tmp'], ['rm', '*']]`
- Nested substitutions are extracted recursively
- Heredocs (`<<EOF`) and here-strings (`<<<`) have their content inlined as a single token
- Quoted strings and escaped spaces are handled correctly, preserving them as single tokens

The return value is a 2-ple with the above list as the 1st element, and the result of `collect_ops` as the 2nd.

The tests below show the full behavior:

In [ ]:
from fastcore.test import test_eq

In [ ]:
def test_split(a, *b, ops=set()): test_eq(extract_commands(a), (list(b), ops))

In [ ]:
test_split('echo <<EOF\nasdf\njkljl\nEOF\n', ['echo', 'asdf\njkljl'])
test_split('echo $(foo)', ['echo', '$(foo)'], ['foo'])
test_split('echo $(foo) | cat -a', ['echo', '$(foo)'], ['foo'], ['cat', '-a'], ops={'|'})
test_split('echo $(cat $(ls))', ['echo', '$(cat $(ls))'], ['cat', '$(ls)'], ['ls'])
test_split('echo "hello world" foo', ['echo', 'hello world', 'foo'])
test_split('echo hello\\ world', ['echo', 'hello world'])
test_split('echo foo; echo bar', ['echo', 'foo'], ['echo', 'bar'], ops={';'})
test_split('echo $HOME "${USER}"', ['echo', '$HOME', '${USER}'])
test_split('sleep 10 &', ['sleep', '10'], ops={';', '&'})
test_split('cat <<< "some text"', ['cat', '<<<', 'some text'])
test_split("echo \"it's a 'test'\"", ['echo', "it's a 'test'"])
test_split('echo "hello $(whoami) there"', ['echo', 'hello $(whoami) there'], ['whoami'])
test_split('echo "path is ${HOME}/bin"', ['echo', 'path is ${HOME}/bin'])
test_split('echo ${arr[0]}', ['echo', '${arr[0]}'])
test_split('echo "$(echo "inner")"', ['echo', '$(echo "inner")'], ['echo', 'inner'])
test_split('echo "$HOME/$(whoami)/file"', ['echo', '$HOME/$(whoami)/file'], ['whoami'])
test_split('echo `whoami`', ['echo', '`whoami`'], ['whoami'])
test_split('(cd /tmp && rm -rf *)', ['cd', '/tmp'], ['rm', '-rf', '*'], ops={'&&'})
test_split('eval "rm -rf /"', ['eval', 'rm -rf /'])
test_split('echo a && echo b || echo c', ['echo', 'a'], ['echo', 'b'], ['echo', 'c'], ops={'&&', '||'})
test_split('cat file > out', ['cat', 'file'], ops={'>'})
test_split('cat file >> out', ['cat', 'file'], ops={'>>'})
test_split('cat < in', ['cat'], ops={'<'})
test_split('diff <(ls dir1) <(ls dir2)',
    ['diff', '<(ls dir1)', '<(ls dir2)'], ['ls', 'dir1'], ['ls', 'dir2'])
test_split('FOO=bar', ops={'='})
test_split('FOO=bar echo hello', ['echo', 'hello'], ops={'='})

In [ ]:
test_split('echo &>file', ['echo'], ops={'&>'})
test_split('echo &>>file', ['echo'], ops={'&>>'})
test_split('echo |& cat', ['echo'], ['cat'], ops={'|&'})

test_split('echo >&2', ['echo'], ops={'>&'})
test_split('cat <&3', ['cat'], ops={'<&'})

## export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()